In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [42]:
from matplotlib import pyplot as plt
from matplotlib import rcParams
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import lightgbm as lgb
import nltk
import numpy as np
from pathlib import Path
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, GlobalMaxPooling1D, Conv1D, Dropout, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.optimizers import Adam
import warnings

In [3]:
rcParams['figure.figsize'] = (16, 8)
plt.style.use('fivethirtyeight')
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
warnings.simplefilter('ignore')

In [6]:
data_dir = Path("data")
feature_dir = Path("data")
val_dir = Path('data')
tst_dir = Path('data')
sub_dir = Path('data')

trn_file = data_dir / 'train.csv'
tst_file = data_dir / 'test_x.csv'
sample_file = data_dir/ 'sample_submission.csv'

target_col = 'author'
n_fold = 5
n_class = 5
seed = 42

In [7]:
algo_name = 'lr'
feature_name = 'tfidf'
model_name = f'{algo_name}_{feature_name}'

feature_file = feature_dir / f'{feature_name}.csv'
p_val_file = val_dir / f'{model_name}.val.csv'
p_tst_file = tst_dir / f'{model_name}.tst.csv'
sub_file = sub_dir / f'{model_name}.csv'


In [8]:
trn = pd.read_csv(trn_file, index_col=0)
print(trn.shape)
trn.head()

(54879, 2)


,text,author
index,,
0,"He was almost choking. There was so much, so m...",3
1,"“Your sister asked for it, I suppose?”",2
2,"She was engaged one day as she walked, in per...",1
3,"The captain was in the porch, keeping himself ...",4
4,"“Have mercy, gentlemen!” odin flung up his han...",3


In [10]:
tst = pd.read_csv(tst_file, index_col=0)
print(tst.shape)
tst.head()

(19617, 1)


,text
index,
0,“Not at all. I think she is one of the most ch...
1,"""No,"" replied he, with sudden consciousness, ""..."
2,As the lady had stated her intention of scream...
3,“And then suddenly in the silence I heard a so...
4,His conviction remained unchanged. So far as I...


In [11]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.stem.snowball import SnowballStemmer

In [12]:
s = trn.text[4]
print(s)

“Have mercy, gentlemen!” odin flung up his hands. “Don’t write that, anyway; have some shame. Here I’ve torn my heart asunder before you, and you seize the opportunity and are fingering the wounds in both halves.... Oh, my God!”


In [17]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\anaconda3\envs\open source sw
[nltk_data]     class\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [18]:
tokens = word_tokenize(s)
print(tokens)

['“', 'Have', 'mercy', ',', 'gentlemen', '!', '”', 'odin', 'flung', 'up', 'his', 'hands', '.', '“', 'Don', '’', 't', 'write', 'that', ',', 'anyway', ';', 'have', 'some', 'shame', '.', 'Here', 'I', '’', 've', 'torn', 'my', 'heart', 'asunder', 'before', 'you', ',', 'and', 'you', 'seize', 'the', 'opportunity', 'and', 'are', 'fingering', 'the', 'wounds', 'in', 'both', 'halves', '....', 'Oh', ',', 'my', 'God', '!', '”']


In [19]:
lemmatizer = WordNetLemmatizer()
[lemmatizer.lemmatize(t) for t in tokens]

['“',
 'Have',
 'mercy',
 ',',
 'gentleman',
 '!',
 '”',
 'odin',
 'flung',
 'up',
 'his',
 'hand',
 '.',
 '“',
 'Don',
 '’',
 't',
 'write',
 'that',
 ',',
 'anyway',
 ';',
 'have',
 'some',
 'shame',
 '.',
 'Here',
 'I',
 '’',
 've',
 'torn',
 'my',
 'heart',
 'asunder',
 'before',
 'you',
 ',',
 'and',
 'you',
 'seize',
 'the',
 'opportunity',
 'and',
 'are',
 'fingering',
 'the',
 'wound',
 'in',
 'both',
 'half',
 '....',
 'Oh',
 ',',
 'my',
 'God',
 '!',
 '”']

In [20]:
stemmer = SnowballStemmer("english")
[stemmer.stem(t) for t in tokens]

['“',
 'have',
 'merci',
 ',',
 'gentlemen',
 '!',
 '”',
 'odin',
 'flung',
 'up',
 'his',
 'hand',
 '.',
 '“',
 'don',
 '’',
 't',
 'write',
 'that',
 ',',
 'anyway',
 ';',
 'have',
 'some',
 'shame',
 '.',
 'here',
 'i',
 '’',
 've',
 'torn',
 'my',
 'heart',
 'asund',
 'befor',
 'you',
 ',',
 'and',
 'you',
 'seiz',
 'the',
 'opportun',
 'and',
 'are',
 'finger',
 'the',
 'wound',
 'in',
 'both',
 'halv',
 '....',
 'oh',
 ',',
 'my',
 'god',
 '!',
 '”']

In [21]:
vec = CountVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 2), min_df=100)
X_cnt = vec.fit_transform(trn['text'])
print(X_cnt.shape)

(54879, 2685)


In [22]:
X_cnt[0, :50].todense()

matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

In [36]:
vec = TfidfVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 3), min_df=50)
X_trn = vec.fit_transform(trn['text'])
X_tst = vec.transform(tst['text'])
print(X.shape, X_tst.shape)

(54879, 5897) (19617, 5897)


In [24]:
X_trn[0, :50].todense()

matrix([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.]])

In [25]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [26]:
y = trn.author.values
y.shape

(54879,)

In [52]:
p_val = np.zeros((trn.shape[0], n_class))
p_tst = np.zeros((tst.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(X_trn, y), 1):
    print(f'training model for CV #{i}')
    clf = lgb.LGBMClassifier(objective='multiclass',
                             n_estimators=1000,
                             num_leaves=64,
                             learning_rate=0.1,
                             min_child_samples=10,
                             subsample=0.7,
                             subsample_freq=1,
                             colsample_bytree=0.7,
                             random_state=seed,
                             n_jobs=-1)
    clf.fit(X_trn[i_trn], y[i_trn],
            eval_set=[(X_trn[i_val], y[i_val])],
            eval_metric='multiclass',
            early_stopping_rounds=10)
    
    p_val[i_val, :] = clf.predict_proba(X_trn[i_val])
    p_tst += clf.predict_proba(X_tst) / n_fold

training model for CV #1
[1]	valid_0's multi_logloss: 1.56224
Training until validation scores don't improve for 10 rounds
[2]	valid_0's multi_logloss: 1.5544
[3]	valid_0's multi_logloss: 1.54705
[4]	valid_0's multi_logloss: 1.53952
[5]	valid_0's multi_logloss: 1.53223
[6]	valid_0's multi_logloss: 1.52512
[7]	valid_0's multi_logloss: 1.51821
[8]	valid_0's multi_logloss: 1.51139
[9]	valid_0's multi_logloss: 1.50501
[10]	valid_0's multi_logloss: 1.49828
[11]	valid_0's multi_logloss: 1.49162
[12]	valid_0's multi_logloss: 1.48538
[13]	valid_0's multi_logloss: 1.47902
[14]	valid_0's multi_logloss: 1.47263
[15]	valid_0's multi_logloss: 1.46654
[16]	valid_0's multi_logloss: 1.46031
[17]	valid_0's multi_logloss: 1.45457
[18]	valid_0's multi_logloss: 1.44871
[19]	valid_0's multi_logloss: 1.44303
[20]	valid_0's multi_logloss: 1.43717
[21]	valid_0's multi_logloss: 1.43149
[22]	valid_0's multi_logloss: 1.42576
[23]	valid_0's multi_logloss: 1.42029
[24]	valid_0's multi_logloss: 1.41479
[25]	valid_0

[212]	valid_0's multi_logloss: 0.950297
[213]	valid_0's multi_logloss: 0.949106
[214]	valid_0's multi_logloss: 0.947962
[215]	valid_0's multi_logloss: 0.946826
[216]	valid_0's multi_logloss: 0.945719
[217]	valid_0's multi_logloss: 0.944561
[218]	valid_0's multi_logloss: 0.943426
[219]	valid_0's multi_logloss: 0.942306
[220]	valid_0's multi_logloss: 0.941194
[221]	valid_0's multi_logloss: 0.940037
[222]	valid_0's multi_logloss: 0.93902
[223]	valid_0's multi_logloss: 0.93797
[224]	valid_0's multi_logloss: 0.93696
[225]	valid_0's multi_logloss: 0.935959
[226]	valid_0's multi_logloss: 0.934848
[227]	valid_0's multi_logloss: 0.933791
[228]	valid_0's multi_logloss: 0.932729
[229]	valid_0's multi_logloss: 0.931694
[230]	valid_0's multi_logloss: 0.930623
[231]	valid_0's multi_logloss: 0.929604
[232]	valid_0's multi_logloss: 0.928633
[233]	valid_0's multi_logloss: 0.927626
[234]	valid_0's multi_logloss: 0.926594
[235]	valid_0's multi_logloss: 0.925615
[236]	valid_0's multi_logloss: 0.924561
[23

[418]	valid_0's multi_logloss: 0.792339
[419]	valid_0's multi_logloss: 0.791832
[420]	valid_0's multi_logloss: 0.791362
[421]	valid_0's multi_logloss: 0.790825
[422]	valid_0's multi_logloss: 0.790348
[423]	valid_0's multi_logloss: 0.789805
[424]	valid_0's multi_logloss: 0.789303
[425]	valid_0's multi_logloss: 0.788824
[426]	valid_0's multi_logloss: 0.788314
[427]	valid_0's multi_logloss: 0.787814
[428]	valid_0's multi_logloss: 0.787302
[429]	valid_0's multi_logloss: 0.786732
[430]	valid_0's multi_logloss: 0.786204
[431]	valid_0's multi_logloss: 0.785647
[432]	valid_0's multi_logloss: 0.785173
[433]	valid_0's multi_logloss: 0.784698
[434]	valid_0's multi_logloss: 0.784237
[435]	valid_0's multi_logloss: 0.783776
[436]	valid_0's multi_logloss: 0.783223
[437]	valid_0's multi_logloss: 0.78273
[438]	valid_0's multi_logloss: 0.782264
[439]	valid_0's multi_logloss: 0.781798
[440]	valid_0's multi_logloss: 0.781343
[441]	valid_0's multi_logloss: 0.780894
[442]	valid_0's multi_logloss: 0.780451
[

[624]	valid_0's multi_logloss: 0.713655
[625]	valid_0's multi_logloss: 0.713386
[626]	valid_0's multi_logloss: 0.713109
[627]	valid_0's multi_logloss: 0.712832
[628]	valid_0's multi_logloss: 0.712601
[629]	valid_0's multi_logloss: 0.712332
[630]	valid_0's multi_logloss: 0.712059
[631]	valid_0's multi_logloss: 0.711794
[632]	valid_0's multi_logloss: 0.711564
[633]	valid_0's multi_logloss: 0.711313
[634]	valid_0's multi_logloss: 0.711054
[635]	valid_0's multi_logloss: 0.710787
[636]	valid_0's multi_logloss: 0.710525
[637]	valid_0's multi_logloss: 0.710261
[638]	valid_0's multi_logloss: 0.709998
[639]	valid_0's multi_logloss: 0.709718
[640]	valid_0's multi_logloss: 0.709442
[641]	valid_0's multi_logloss: 0.709175
[642]	valid_0's multi_logloss: 0.708923
[643]	valid_0's multi_logloss: 0.708661
[644]	valid_0's multi_logloss: 0.70841
[645]	valid_0's multi_logloss: 0.708164
[646]	valid_0's multi_logloss: 0.707924
[647]	valid_0's multi_logloss: 0.707676
[648]	valid_0's multi_logloss: 0.707415
[

[830]	valid_0's multi_logloss: 0.671046
[831]	valid_0's multi_logloss: 0.67091
[832]	valid_0's multi_logloss: 0.670764
[833]	valid_0's multi_logloss: 0.670593
[834]	valid_0's multi_logloss: 0.670425
[835]	valid_0's multi_logloss: 0.670304
[836]	valid_0's multi_logloss: 0.670162
[837]	valid_0's multi_logloss: 0.670023
[838]	valid_0's multi_logloss: 0.66987
[839]	valid_0's multi_logloss: 0.669729
[840]	valid_0's multi_logloss: 0.669609
[841]	valid_0's multi_logloss: 0.669458
[842]	valid_0's multi_logloss: 0.669311
[843]	valid_0's multi_logloss: 0.66914
[844]	valid_0's multi_logloss: 0.668997
[845]	valid_0's multi_logloss: 0.668871
[846]	valid_0's multi_logloss: 0.668747
[847]	valid_0's multi_logloss: 0.668612
[848]	valid_0's multi_logloss: 0.668484
[849]	valid_0's multi_logloss: 0.668322
[850]	valid_0's multi_logloss: 0.668182
[851]	valid_0's multi_logloss: 0.668009
[852]	valid_0's multi_logloss: 0.667878
[853]	valid_0's multi_logloss: 0.667754
[854]	valid_0's multi_logloss: 0.66759
[855

...

[134]	valid_0's multi_logloss: 1.05994
[135]	valid_0's multi_logloss: 1.05798
[136]	valid_0's multi_logloss: 1.05606
[137]	valid_0's multi_logloss: 1.05427
[138]	valid_0's multi_logloss: 1.05241
[139]	valid_0's multi_logloss: 1.05062
[140]	valid_0's multi_logloss: 1.04867
[141]	valid_0's multi_logloss: 1.04682
[142]	valid_0's multi_logloss: 1.04503
[143]	valid_0's multi_logloss: 1.04329
[144]	valid_0's multi_logloss: 1.04151
[145]	valid_0's multi_logloss: 1.03959
[146]	valid_0's multi_logloss: 1.03785
[147]	valid_0's multi_logloss: 1.03605
[148]	valid_0's multi_logloss: 1.03425
[149]	valid_0's multi_logloss: 1.03256
[150]	valid_0's multi_logloss: 1.03087
[151]	valid_0's multi_logloss: 1.02916
[152]	valid_0's multi_logloss: 1.02751
[153]	valid_0's multi_logloss: 1.02595
[154]	valid_0's multi_logloss: 1.02416
[155]	valid_0's multi_logloss: 1.0225
[156]	valid_0's multi_logloss: 1.02078
[157]	valid_0's multi_logloss: 1.01912
[158]	valid_0's multi_logloss: 1.01757
[159]	valid_0's multi_logl

[341]	valid_0's multi_logloss: 0.831324
[342]	valid_0's multi_logloss: 0.830655
[343]	valid_0's multi_logloss: 0.829998
[344]	valid_0's multi_logloss: 0.829301
[345]	valid_0's multi_logloss: 0.828624
[346]	valid_0's multi_logloss: 0.827973
[347]	valid_0's multi_logloss: 0.827295
[348]	valid_0's multi_logloss: 0.826662
[349]	valid_0's multi_logloss: 0.826026
[350]	valid_0's multi_logloss: 0.825396
[351]	valid_0's multi_logloss: 0.824728
[352]	valid_0's multi_logloss: 0.824047
[353]	valid_0's multi_logloss: 0.823381
[354]	valid_0's multi_logloss: 0.822696
[355]	valid_0's multi_logloss: 0.822087
[356]	valid_0's multi_logloss: 0.821456
[357]	valid_0's multi_logloss: 0.820832
[358]	valid_0's multi_logloss: 0.820235
[359]	valid_0's multi_logloss: 0.819591
[360]	valid_0's multi_logloss: 0.818991
[361]	valid_0's multi_logloss: 0.81836
[362]	valid_0's multi_logloss: 0.817699
[363]	valid_0's multi_logloss: 0.817079
[364]	valid_0's multi_logloss: 0.816434
[365]	valid_0's multi_logloss: 0.815834
[

[547]	valid_0's multi_logloss: 0.730933
[548]	valid_0's multi_logloss: 0.730598
[549]	valid_0's multi_logloss: 0.730229
[550]	valid_0's multi_logloss: 0.729866
[551]	valid_0's multi_logloss: 0.729572
[552]	valid_0's multi_logloss: 0.729238
[553]	valid_0's multi_logloss: 0.728919
[554]	valid_0's multi_logloss: 0.728586
[555]	valid_0's multi_logloss: 0.728319
[556]	valid_0's multi_logloss: 0.727947
[557]	valid_0's multi_logloss: 0.72761
[558]	valid_0's multi_logloss: 0.727284
[559]	valid_0's multi_logloss: 0.726934
[560]	valid_0's multi_logloss: 0.726566
[561]	valid_0's multi_logloss: 0.726264
[562]	valid_0's multi_logloss: 0.725958
[563]	valid_0's multi_logloss: 0.725643
[564]	valid_0's multi_logloss: 0.725334
[565]	valid_0's multi_logloss: 0.724965
[566]	valid_0's multi_logloss: 0.724648
[567]	valid_0's multi_logloss: 0.724343
[568]	valid_0's multi_logloss: 0.724026
[569]	valid_0's multi_logloss: 0.723719
[570]	valid_0's multi_logloss: 0.723363
[571]	valid_0's multi_logloss: 0.723086
[

[753]	valid_0's multi_logloss: 0.678355
[754]	valid_0's multi_logloss: 0.678169
[755]	valid_0's multi_logloss: 0.677982
[756]	valid_0's multi_logloss: 0.677805
[757]	valid_0's multi_logloss: 0.67759
[758]	valid_0's multi_logloss: 0.677414
[759]	valid_0's multi_logloss: 0.677206
[760]	valid_0's multi_logloss: 0.677023
[761]	valid_0's multi_logloss: 0.676838
[762]	valid_0's multi_logloss: 0.676665
[763]	valid_0's multi_logloss: 0.676489
[764]	valid_0's multi_logloss: 0.676277
[765]	valid_0's multi_logloss: 0.676107
[766]	valid_0's multi_logloss: 0.675931
[767]	valid_0's multi_logloss: 0.675744
[768]	valid_0's multi_logloss: 0.675552
[769]	valid_0's multi_logloss: 0.675383
[770]	valid_0's multi_logloss: 0.675193
[771]	valid_0's multi_logloss: 0.674998
[772]	valid_0's multi_logloss: 0.674816
[773]	valid_0's multi_logloss: 0.674601
[774]	valid_0's multi_logloss: 0.674434
[775]	valid_0's multi_logloss: 0.674264
[776]	valid_0's multi_logloss: 0.674079
[777]	valid_0's multi_logloss: 0.673891
[

[959]	valid_0's multi_logloss: 0.648156
[960]	valid_0's multi_logloss: 0.648042
[961]	valid_0's multi_logloss: 0.647918
[962]	valid_0's multi_logloss: 0.647809
[963]	valid_0's multi_logloss: 0.647666
[964]	valid_0's multi_logloss: 0.64756
[965]	valid_0's multi_logloss: 0.647431
[966]	valid_0's multi_logloss: 0.647323
[967]	valid_0's multi_logloss: 0.64723
[968]	valid_0's multi_logloss: 0.647131
[969]	valid_0's multi_logloss: 0.64703
[970]	valid_0's multi_logloss: 0.646926
[971]	valid_0's multi_logloss: 0.646812
[972]	valid_0's multi_logloss: 0.646706
[973]	valid_0's multi_logloss: 0.646605
[974]	valid_0's multi_logloss: 0.6465
[975]	valid_0's multi_logloss: 0.646392
[976]	valid_0's multi_logloss: 0.646293
[977]	valid_0's multi_logloss: 0.646191
[978]	valid_0's multi_logloss: 0.646088
[979]	valid_0's multi_logloss: 0.645969
[980]	valid_0's multi_logloss: 0.64581
[981]	valid_0's multi_logloss: 0.645704
[982]	valid_0's multi_logloss: 0.645578
[983]	valid_0's multi_logloss: 0.645492
[984]	

In [53]:
print(f'Accuracy (CV): {accuracy_score(y, np.argmax(p_val, axis=1)) * 100:8.4f}%')
print(f'Log Loss (CV): {log_loss(pd.get_dummies(y), p_val):8.4f}')

Accuracy (CV):  76.1439%
Log Loss (CV):   0.6528


In [54]:
np.savetxt(p_val_file, p, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, p_tst, fmt='%.6f', delimiter=',')

In [55]:
sub = pd.read_csv(sample_file, index_col=0)
print(sub.shape)
sub.head()

(19617, 5)


,0,1,2,3,4
index,,,,,
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0


In [56]:
sub[sub.columns] = p_tst
sub.head()

,0,1,2,3,4
index,,,,,
0,0.0460,0.7333,0.1628,0.0472,0.0108
1,0.0268,0.8369,0.0208,0.0371,0.0784
2,0.9395,0.0151,0.0183,0.0069,0.0201
3,0.0188,0.0285,0.9049,0.0327,0.0151
4,0.4065,0.1308,0.2014,0.1216,0.1398


In [57]:
sub.to_csv(sub_file)